**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [26]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense, Dropout
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten

# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

> `epsilon` is the $\epsilon$-greedy chosen to allow more exploration during training and avoid model's overfitting. In other words, a small/large value of epsilon favors exploitation/exploration.

> In fact, `epsilon` is the fraction of times in which one selects a completely random action during training. In the remaining times, the model will perform the action with the highest Q-value, which makes it more likely to stay close to a local maximum (around which Q-values tend to be higher for many types of problems).

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=50  # ?? # set small when debugging
epochs_test=50  # ?? # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

> The `position` is an array that contains information about the current position of the rat (the only cell with a $1$) and the position of the border cells (or neighbor to a border cell), which have $-1$s.

> The `board` is the reward that the rat will receive if he goes to a cell. It is $0.5$ in the cells with cheese and $1$ in the poisoned cells. It gets updated to $0$ once the rat arrives in a cell.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(4)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        state = env.reset()
        game_over = False

        win, lose = 0, 0
        
        while not game_over:
            action = agent.act(state, train=False)
            state, reward, game_over = env.act(action)
            if reward > 0:
                win += reward
            elif reward < 0:
                lose -= reward 
         
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))
    return score/epochs

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 10.0/14.0. Average score (-4.0)
Win/lose count 12.0/20.0. Average score (-6.0)
Win/lose count 8.0/13.0. Average score (-5.666666666666667)
Win/lose count 10.0/11.0. Average score (-4.5)
Win/lose count 15.5/13.0. Average score (-3.1)
Win/lose count 8.5/12.0. Average score (-3.1666666666666665)
Win/lose count 13.5/13.0. Average score (-2.642857142857143)
Win/lose count 14.5/17.0. Average score (-2.625)
Win/lose count 6.5/16.0. Average score (-3.388888888888889)
Win/lose count 8.5/10.0. Average score (-3.2)
Win/lose count 12.5/13.0. Average score (-2.9545454545454546)
Win/lose count 14.5/17.0. Average score (-2.9166666666666665)
Win/lose count 15.5/21.0. Average score (-3.1153846153846154)
Win/lose count 9.0/16.0. Average score (-3.392857142857143)
Win/lose count 11.5/14.0. Average score (-3.3333333333333335)
Win/lose count 10.0/6.0. Average score (-2.875)
Win/lose count 5.5/13.0. Average score (-3.1470588235294117)
Win/lose count 9.5/15.0. Average score (-3.277777777777777

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




**Answer:**

_Part 1_

We start from the definition to prove the first equation:

\begin{align}
Q^{\pi}(s,a)&=E[ \sum_{k=0}^{\infty} \gamma^k r_{t+k}|s_t = s, a_t = a] \\
& = E[ r(s,a) + \sum_{k=1}^{\infty} \gamma^k r_{t+k}|s_t = s, a_t = a] \\
& = r(s,a) + \gamma E[ \sum_{k=0}^{\infty} \gamma^k r_{t+1+k}|s_t = s, a_t = a] \\
& = r(s,a) + \gamma \sum_{(s',a')}(p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a )E[\sum_{k=0}^{\infty}\gamma^k r_{t+1+k}|s_{t+1}=s',a_{t+1}=a']\\
&=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{align}

_Part 2_

We assume the existence of the optimal policy $\pi$ and we look for the Bellman equation for $Q^{*} = Q^{\pi^{*}}$. To do that, we use the Bellman equation we just derived.

\begin{align}
Q^{*}(s,a) &= \max_{\pi}E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] \\
&= r(s,a) + \gamma \max_{\pi} \sum(p(s_{t+1}=s' |s_t = s, a_t = a )Q^{\pi}(s',a')
\end{align}

We now exchange the sum and the maximum. The proof comes directly after proving the following two inequalities:

> 1 - $\max_{\pi} \sum(p(s_{t+1}=s' |s_t = s, a_t = a )Q^{\pi}(s',a') \leq \sum(p(s_{t+1}=s' |s_t = s, a_t = a ) \max_{\pi} Q^{\pi}(s',a')$ (trivial)

> 2 - $\max_{\pi} \sum(p(s_{t+1}=s' |s_t = s, a_t = a )Q^{\pi}(s',a') \geq \sum(p(s_{t+1}=s' |s_t = s, a_t = a ) \max_{\pi} Q^{\pi}(s',a')$

> _Proof_: Consider $\hat{\pi}(s',a') = \arg\max_{\pi'} Q^{\pi'}(s',a')$, which is the policy that maximizes $Q^{\pi'}(s',a')$. Consider the policy $ \pi'$ that reproduces $\hat{\pi}(s',a')$ once the rat arrives at $s'$ and performs action $a'$. Then

\begin{align}
\sum(p(s_{t+1}=s' |s_t = s, a_t = a ) \max_{\pi} Q^{\pi}(s',a') &= \sum(p(s_{t+1}=s' |s_t = s, a_t = a ) Q^{\hat{\pi}(s', a')}(s',a') \\
&= \sum(p(s_{t+1}=s' |s_t = s, a_t = a ) Q^{\pi'}(s',a')) \\
&\leq \max_{\pi} \sum(p(s_{t+1}=s' |s_t = s, a_t = a )Q^{\pi}(s',a')
\end{align}


We exchange maximum and sum and continue working on the equation:
\begin{align}
Q^{*}(s,a) &= r(s,a) + \gamma  \sum(p(s_{t+1}=s' |s_t = s, a_t = a )\max_{\pi, a'} Q^{\pi}(s',a') \\
&= r(s,a) + \gamma \sum(p(s_{t+1}=s' |s_t = s, a_t = a )\max_{a'}\max_{\pi} Q^{\pi}(s',a') \\ 
&= r(s,a) + \gamma \sum(p(s_{t+1}=s' |s_t = s, a_t = a )\max_{a'}Q^{*}(s',a') \\ 
&=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{align}

_Part 3_

The objective

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

enforces the just derived Bellman equation in that the optimal policy would have a zero (and minimum) loss. In other words, if the learned policy converges to the optimal policy than the objective function will converge to zero.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory) > self.max_memory:
            idx = np.random.randint(len(self.memory))
            del self.memory[idx]

    def random_access(self):
        idx = np.random.randint(len(self.memory))
        return self.memory[idx]

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape([1, *s.shape]))[0,:])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            s_, n_s_, a_, r_, game_over_  = self.memory.random_access()
            
            target_q[i] = self.model.predict(s_.reshape([1, *s_.shape]))[0]            
            if game_over_:
                ######## FILL IN
                input_states[i] = s_
                target_q[i,a_] = r_
            else:
                ######## FILL IN
                input_states[i] = s_
                target_q[i, a_] = r_  + self.discount * max(self.model.predict(n_s_.reshape([1, *s_.shape]))[0])
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1, hidden_size=30, extra_layer=False, **kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        # NN Model
        model = Sequential()
        model.add(Flatten(input_shape=(5, 5, self.n_state,)))
        model.add(Dense(hidden_size, activation ='relu'))
        if extra_layer:
            model.add(Dense(hidden_size, activation ='relu'))
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/050 | Loss 0.0088 | Win/lose count 2.0/2.0 (0.0)
Epoch 001/050 | Loss 0.0158 | Win/lose count 5.5/2.0 (3.5)
Epoch 002/050 | Loss 0.0150 | Win/lose count 5.5/5.0 (0.5)
Epoch 003/050 | Loss 0.0059 | Win/lose count 4.0/4.0 (0.0)
Epoch 004/050 | Loss 0.0078 | Win/lose count 5.0/3.0 (2.0)
Epoch 005/050 | Loss 0.0062 | Win/lose count 2.5/2.0 (0.5)
Epoch 006/050 | Loss 0.0028 | Win/lose count 1.0/5.0 (-4.0)
Epoch 007/050 | Loss 0.0124 | Win/lose count 1.5/3.0 (-1.5)
Epoch 008/050 | Loss 0.0494 | Win/lose count 1.5/3.0 (-1.5)
Epoch 009/050 | Loss 0.1071 | Win/lose count 1.5/3.0 (-1.5)
Epoch 010/050 | Loss 0.0142 | Win/lose count 2.5/5.0 (-2.5)
Epoch 011/050 | Loss 0.0026 | Win/lose count 1.5/2.0 (-0.5)
Epoch 012/050 | Loss 0.0618 | Win/lose count 1.5/4.0 (-2.5)
Epoch 013/050 | Loss 0.0047 | Win/lose count 4.0/5.0 (-1.0)
Epoch 014/050 | Loss 0.0045 | Win/lose count 3.5/2.0 (1.5)
Epoch 015/050 | Loss 0.0059 | Win/lose count 3.5/2.0 (1.5)
Epoch 016/050 | Loss 0.0595 | Win/lose count 5.0

### Studying multiple architectures/hyperparameters with fully connected layers

In [33]:
results = []

for epsilon in [0.1]:
    for lr in [.1]:
        for hidden_size in [30, 50]:
            for extra_layer in [False, True]:
                # training
                env = Environment(grid_size=size, max_time=T, temperature=0.3)
                agent = DQN_FC(size, lr=lr, epsilon = epsilon, memory_size=2000, batch_size = 32, hidden_size=hidden_size, extra_layer=extra_layer)
                train(agent, env, epochs_train, prefix='fc_train')
                # testing
                env = Environment(grid_size=size, max_time=T, temperature=0.3)
                score = test(agent, env, epochs_test, prefix='fc_test')

                result = {'epsilon': epsilon,
                          'lr': lr,
                          'hidden_size': hidden_size,
                          'extra_layer': extra_layer,
                          'score': score
                         }
                print('saving result: {}'.format(result))
                results.append(result)

Epoch 000/050 | Loss 0.0181 | Win/lose count 5.5/4.0 (1.5)
Epoch 001/050 | Loss 0.0043 | Win/lose count 2.5/4.0 (-1.5)
Epoch 002/050 | Loss 0.0144 | Win/lose count 1.5/4.0 (-2.5)
Epoch 003/050 | Loss 0.0066 | Win/lose count 4.5/4.0 (0.5)
Epoch 004/050 | Loss 0.0124 | Win/lose count 4.0/1.0 (3.0)
Epoch 005/050 | Loss 0.0082 | Win/lose count 3.5/5.0 (-1.5)
Epoch 006/050 | Loss 0.0052 | Win/lose count 12.0/10.0 (2.0)
Epoch 007/050 | Loss 0.0095 | Win/lose count 2.0/3.0 (-1.0)
Epoch 008/050 | Loss 0.0034 | Win/lose count 11.5/9.0 (2.5)
Epoch 009/050 | Loss 0.0062 | Win/lose count 6.0/3.0 (3.0)
Epoch 010/050 | Loss 0.0045 | Win/lose count 4.5/0 (4.5)
Epoch 011/050 | Loss 0.0572 | Win/lose count 3.5/1.0 (2.5)
Epoch 012/050 | Loss 0.0066 | Win/lose count 3.0/2.0 (1.0)
Epoch 013/050 | Loss 0.0084 | Win/lose count 3.0/1.0 (2.0)
Epoch 014/050 | Loss 0.0049 | Win/lose count 6.5/3.0 (3.5)
Epoch 015/050 | Loss 0.0069 | Win/lose count 2.0/1.0 (1.0)
Epoch 016/050 | Loss 0.0100 | Win/lose count 6.5/2.

Epoch 043/050 | Loss 0.0024 | Win/lose count 9.0/4.0 (5.0)
Epoch 044/050 | Loss 0.0058 | Win/lose count 6.0/0 (6.0)
Epoch 045/050 | Loss 0.0042 | Win/lose count 7.5/1.0 (6.5)
Epoch 046/050 | Loss 0.0045 | Win/lose count 4.0/2.0 (2.0)
Epoch 047/050 | Loss 0.0118 | Win/lose count 6.0/3.0 (3.0)
Epoch 048/050 | Loss 0.0054 | Win/lose count 9.0/2.0 (7.0)
Epoch 049/050 | Loss 0.0569 | Win/lose count 3.5/2.0 (1.5)
Win/lose count 1.0/0. Average score (1.0)
Win/lose count 0.5/0. Average score (0.75)
Win/lose count 0.5/0. Average score (0.6666666666666666)
Win/lose count 1.0/0. Average score (0.75)
Win/lose count 0/0. Average score (0.6)
Win/lose count 4.5/0. Average score (1.25)
Win/lose count 2.5/0. Average score (1.4285714285714286)
Win/lose count 2.0/0. Average score (1.5)
Win/lose count 3.5/0. Average score (1.7222222222222223)
Win/lose count 0.5/0. Average score (1.6)
Win/lose count 2.0/0. Average score (1.6363636363636365)
Win/lose count 2.5/0. Average score (1.7083333333333333)
Win/lose 

Win/lose count 1.5/0. Average score (3.9047619047619047)
Win/lose count 6.0/0. Average score (3.953488372093023)
Win/lose count 2.0/0. Average score (3.909090909090909)
Win/lose count 3.0/0. Average score (3.888888888888889)
Win/lose count 2.0/1.0. Average score (3.8260869565217392)
Win/lose count 2.5/0. Average score (3.797872340425532)
Win/lose count 6.0/0. Average score (3.84375)
Win/lose count 7.0/0. Average score (3.9081632653061225)
Win/lose count 4.5/0. Average score (3.92)
Final score: 3.92
saving result: {'epsilon': 0.1, 'lr': 0.1, 'hidden_size': 50, 'extra_layer': False, 'score': 3.92}
Epoch 000/050 | Loss 0.0006 | Win/lose count 1.0/6.0 (-5.0)
Epoch 001/050 | Loss 0.0147 | Win/lose count 4.5/8.0 (-3.5)
Epoch 002/050 | Loss 0.0033 | Win/lose count 2.5/2.0 (0.5)
Epoch 003/050 | Loss 0.0079 | Win/lose count 2.5/9.0 (-6.5)
Epoch 004/050 | Loss 0.0096 | Win/lose count 4.0/5.0 (-1.0)
Epoch 005/050 | Loss 0.0084 | Win/lose count 4.0/1.0 (3.0)
Epoch 006/050 | Loss 0.0030 | Win/lose 

In [34]:
import pandas as pd

results_df = pd.DataFrame(results)

results_df.head()

,epsilon,extra_layer,hidden_size,lr,score
0,0.1,False,30,0.1,2.63
1,0.1,True,30,0.1,1.86
2,0.1,False,50,0.1,3.92
3,0.1,True,50,0.1,2.79


**Analysis**

We observed better results for a neural network with a single fully-connected layer and 50 hidden nodes.

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [27]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1, hidden_sizes=(50, 30), add_dropout=False, **kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(hidden_sizes[0], (2,2), input_shape=(5,5,self.n_state,), activation='relu'))
        model.add(Conv2D(hidden_sizes[1], (2,2), activation='relu'))
        model.add(Flatten())
        if add_dropout:
            model.add(Dropout(0.5))
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [23]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/050 | Loss 0.0008 | Win/lose count 1.0/1.0 (0.0)
Epoch 001/050 | Loss 0.0117 | Win/lose count 4.5/11.0 (-6.5)
Epoch 002/050 | Loss 0.0064 | Win/lose count 8.0/3.0 (5.0)
Epoch 003/050 | Loss 0.0051 | Win/lose count 5.0/3.0 (2.0)
Epoch 004/050 | Loss 0.0029 | Win/lose count 7.5/4.0 (3.5)
Epoch 005/050 | Loss 0.0031 | Win/lose count 4.5/2.0 (2.5)
Epoch 006/050 | Loss 0.0020 | Win/lose count 11.0/0 (11.0)
Epoch 007/050 | Loss 0.0069 | Win/lose count 5.0/2.0 (3.0)
Epoch 008/050 | Loss 0.0021 | Win/lose count 13.0/2.0 (11.0)
Epoch 009/050 | Loss 0.0020 | Win/lose count 8.5/3.0 (5.5)
Epoch 010/050 | Loss 0.0038 | Win/lose count 14.0/3.0 (11.0)
Epoch 011/050 | Loss 0.0008 | Win/lose count 11.5/3.0 (8.5)
Epoch 012/050 | Loss 0.0027 | Win/lose count 12.0/1.0 (11.0)
Epoch 013/050 | Loss 0.0084 | Win/lose count 2.5/3.0 (-0.5)
Epoch 014/050 | Loss 0.0012 | Win/lose count 7.0/1.0 (6.0)
Epoch 015/050 | Loss 0.0011 | Win/lose count 10.5/4.0 (6.5)
Epoch 016/050 | Loss 0.0488 | Win/lose count 

### Studying multiple architectures/hyperparameters with convolutional layers

In [30]:
results = []

for epsilon in [0.1]:
    for lr in [.1]:
        for hidden_sizes in [(50, 30), (100, 50)]:
            for add_dropout in [False, True]:
                # training
                env = Environment(grid_size=size, max_time=T, temperature=0.3)
                agent = DQN_CNN(size, lr=lr, epsilon = epsilon, memory_size=2000, batch_size = 32, hidden_sizes=hidden_sizes, add_dropout=add_dropout)
                train(agent, env, epochs_train, prefix='cnn_train')
                # testing
                env = Environment(grid_size=size, max_time=T, temperature=0.3)
                score = test(agent, env, epochs_test, prefix='fc_test')

                result = {'epsilon': epsilon,
                          'lr': lr,
                          'hidden_sizes': hidden_sizes,
                          'add_dropout': add_dropout,
                          'score': score
                         }
                print('saving result: {}'.format(result))
                results.append(result)

Epoch 000/050 | Loss 0.0132 | Win/lose count 4.0/4.0 (0.0)
Epoch 001/050 | Loss 0.0051 | Win/lose count 2.0/3.0 (-1.0)
Epoch 002/050 | Loss 0.0173 | Win/lose count 2.5/2.0 (0.5)
Epoch 003/050 | Loss 0.0014 | Win/lose count 2.0/4.0 (-2.0)
Epoch 004/050 | Loss 0.0037 | Win/lose count 4.0/4.0 (0.0)
Epoch 005/050 | Loss 0.0221 | Win/lose count 4.5/5.0 (-0.5)
Epoch 006/050 | Loss 0.0058 | Win/lose count 6.5/7.0 (-0.5)
Epoch 007/050 | Loss 0.0023 | Win/lose count 7.0/2.0 (5.0)
Epoch 008/050 | Loss 0.0043 | Win/lose count 4.0/1.0 (3.0)
Epoch 009/050 | Loss 0.0656 | Win/lose count 5.5/0 (5.5)
Epoch 010/050 | Loss 0.0569 | Win/lose count 2.5/4.0 (-1.5)
Epoch 011/050 | Loss 0.0030 | Win/lose count 12.5/6.0 (6.5)
Epoch 012/050 | Loss 0.0040 | Win/lose count 8.0/3.0 (5.0)
Epoch 013/050 | Loss 0.0048 | Win/lose count 15.5/6.0 (9.5)
Epoch 014/050 | Loss 0.0014 | Win/lose count 11.5/2.0 (9.5)
Epoch 015/050 | Loss 0.0024 | Win/lose count 7.0/5.0 (2.0)
Epoch 016/050 | Loss 0.0027 | Win/lose count 14.5/

Epoch 043/050 | Loss 0.0138 | Win/lose count 3.0/0 (3.0)
Epoch 044/050 | Loss 0.0031 | Win/lose count 11.0/1.0 (10.0)
Epoch 045/050 | Loss 0.0057 | Win/lose count 5.5/4.0 (1.5)
Epoch 046/050 | Loss 0.0067 | Win/lose count 1.5/3.0 (-1.5)
Epoch 047/050 | Loss 0.0093 | Win/lose count 4.5/6.0 (-1.5)
Epoch 048/050 | Loss 0.0070 | Win/lose count 3.5/4.0 (-0.5)
Epoch 049/050 | Loss 0.0056 | Win/lose count 3.5/2.0 (1.5)
Win/lose count 0/0. Average score (0.0)
Win/lose count 1.0/0. Average score (0.5)
Win/lose count 0.5/0. Average score (0.5)
Win/lose count 1.5/0. Average score (0.75)
Win/lose count 0.5/0. Average score (0.7)
Win/lose count 3.0/2.0. Average score (0.75)
Win/lose count 4.0/0. Average score (1.2142857142857142)
Win/lose count 2.5/1.0. Average score (1.25)
Win/lose count 2.0/0. Average score (1.3333333333333333)
Win/lose count 2.5/0. Average score (1.45)
Win/lose count 3.5/0. Average score (1.6363636363636365)
Win/lose count 4.0/0. Average score (1.8333333333333333)
Win/lose count

Win/lose count 9.5/0. Average score (6.670731707317073)
Win/lose count 8.0/0. Average score (6.7023809523809526)
Win/lose count 8.5/0. Average score (6.744186046511628)
Win/lose count 1.0/0. Average score (6.613636363636363)
Win/lose count 9.0/0. Average score (6.666666666666667)
Win/lose count 10.5/0. Average score (6.75)
Win/lose count 5.0/0. Average score (6.712765957446808)
Win/lose count 8.5/0. Average score (6.75)
Win/lose count 4.0/0. Average score (6.6938775510204085)
Win/lose count 11.0/0. Average score (6.78)
Final score: 6.78
saving result: {'epsilon': 0.1, 'lr': 0.1, 'hidden_sizes': (100, 50), 'add_dropout': False, 'score': 6.78}
Epoch 000/050 | Loss 0.0094 | Win/lose count 5.5/11.0 (-5.5)
Epoch 001/050 | Loss 0.0051 | Win/lose count 2.5/2.0 (0.5)
Epoch 002/050 | Loss 0.0096 | Win/lose count 8.0/2.0 (6.0)
Epoch 003/050 | Loss 0.0070 | Win/lose count 3.0/4.0 (-1.0)
Epoch 004/050 | Loss 0.0143 | Win/lose count 5.0/3.0 (2.0)
Epoch 005/050 | Loss 0.0021 | Win/lose count 2.5/2.0

In [32]:
results_df = pd.DataFrame(results)

results_df.head()

,add_dropout,epsilon,hidden_sizes,lr,score
0,False,0.1,"(50, 30)",0.1,5.43
1,True,0.1,"(50, 30)",0.1,1.75
2,False,0.1,"(100, 50)",0.1,6.78
3,True,0.1,"(100, 50)",0.1,9.82


**Analysis**

We observed better results for a larger neural network using dropout before the last fully-connected layer.

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [39]:
results = []
for epsilon in [0.1]:
    for lr in [0.1]:
        for temperature in [0.1 * n for n in range(1, 10)]:
            env = Environment(grid_size=size, max_time=T,temperature=temperature)
            agent_cnn = DQN_CNN(size, lr=lr, epsilon = epsilon, memory_size=2000, batch_size = 32, hidden_sizes=(100, 50), add_dropout=True)
            train(agent_cnn, env, epochs_train, prefix='cnn_train')
            score = test(agent_cnn,env,epochs_test,prefix='cnn_test')

            result = {
                'architecture': 'cnn',
                'temperature': temperature,
                'score': score
            }
            print('saving result {}'.format(result))
            results.append(result)

for epsilon in [0.1]:
    for lr in [0.1]:
        for temperature in [0.1 * n for n in range(1, 10)]:
            env = Environment(grid_size=size, max_time=T,temperature=temperature)
            agent_cnn = DQN_FC(size, lr=lr, epsilon = epsilon, memory_size=2000, batch_size = 32)
            train(agent_cnn, env, epochs_train, prefix='fc_train')
            score = test(agent_cnn,env,epochs_test,prefix='fc_test')

            result = {
                'architecture': 'fc',
                'temperature': temperature,
                'score': score
            }
            print('saving result {}'.format(result))
            results.append(result)

Epoch 000/050 | Loss 0.0003 | Win/lose count 0/0 (0)
Epoch 001/050 | Loss 0.0002 | Win/lose count 1.0/2.0 (-1.0)
Epoch 002/050 | Loss 0.0002 | Win/lose count 1.0/1.0 (0.0)
Epoch 003/050 | Loss 0.0001 | Win/lose count 1.0/0 (1.0)
Epoch 004/050 | Loss 0.0019 | Win/lose count 1.0/1.0 (0.0)
Epoch 005/050 | Loss 0.0002 | Win/lose count 1.0/1.0 (0.0)
Epoch 006/050 | Loss 0.0001 | Win/lose count 1.5/1.0 (0.5)
Epoch 007/050 | Loss 0.0001 | Win/lose count 0/0 (0)
Epoch 008/050 | Loss 0.0001 | Win/lose count 1.0/1.0 (0.0)
Epoch 009/050 | Loss 0.0000 | Win/lose count 1.0/0 (1.0)
Epoch 010/050 | Loss 0.0021 | Win/lose count 0.5/2.0 (-1.5)
Epoch 011/050 | Loss 0.0072 | Win/lose count 1.0/0 (1.0)
Epoch 012/050 | Loss 0.0074 | Win/lose count 0/0 (0)
Epoch 013/050 | Loss 0.0000 | Win/lose count 1.0/1.0 (0.0)
Epoch 014/050 | Loss 0.0002 | Win/lose count 0/1.0 (-1.0)
Epoch 015/050 | Loss 0.0002 | Win/lose count 1.5/2.0 (-0.5)
Epoch 016/050 | Loss 0.0001 | Win/lose count 2.5/1.0 (1.5)
Epoch 017/050 | Los

Epoch 043/050 | Loss 0.0022 | Win/lose count 2.0/2.0 (0.0)
Epoch 044/050 | Loss 0.0016 | Win/lose count 4.5/2.0 (2.5)
Epoch 045/050 | Loss 0.0005 | Win/lose count 3.0/1.0 (2.0)
Epoch 046/050 | Loss 0.0033 | Win/lose count 1.5/1.0 (0.5)
Epoch 047/050 | Loss 0.0035 | Win/lose count 2.5/1.0 (1.5)
Epoch 048/050 | Loss 0.0004 | Win/lose count 1.0/1.0 (0.0)
Epoch 049/050 | Loss 0.0003 | Win/lose count 3.0/2.0 (1.0)
Win/lose count 1.5/0. Average score (1.5)
Win/lose count 4.0/1.0. Average score (2.25)
Win/lose count 1.5/0. Average score (2.0)
Win/lose count 0.5/0. Average score (1.625)
Win/lose count 0.5/0. Average score (1.4)
Win/lose count 3.5/0. Average score (1.75)
Win/lose count 5.0/0. Average score (2.2142857142857144)
Win/lose count 1.0/0. Average score (2.0625)
Win/lose count 0.5/0. Average score (1.8888888888888888)
Win/lose count 2.0/0. Average score (1.9)
Win/lose count 2.0/0. Average score (1.9090909090909092)
Win/lose count 2.0/0. Average score (1.9166666666666667)
Win/lose count

Win/lose count 6.0/1.0. Average score (3.9047619047619047)
Win/lose count 4.0/0. Average score (3.9069767441860463)
Win/lose count 0.5/0. Average score (3.8295454545454546)
Win/lose count 2.5/1.0. Average score (3.7777777777777777)
Win/lose count 7.5/1.0. Average score (3.8369565217391304)
Win/lose count 8.0/0. Average score (3.925531914893617)
Win/lose count 1.5/0. Average score (3.875)
Win/lose count 4.5/1.0. Average score (3.86734693877551)
Win/lose count 5.0/0. Average score (3.89)
Final score: 3.89
saving result {'architecture': 'cnn', 'temperature': 0.30000000000000004, 'score': 3.89}
Epoch 000/050 | Loss 0.0044 | Win/lose count 5.0/4.0 (1.0)
Epoch 001/050 | Loss 0.0085 | Win/lose count 4.0/4.0 (0.0)
Epoch 002/050 | Loss 0.0021 | Win/lose count 2.5/4.0 (-1.5)
Epoch 003/050 | Loss 0.0105 | Win/lose count 10.5/9.0 (1.5)
Epoch 004/050 | Loss 0.0059 | Win/lose count 2.0/2.0 (0.0)
Epoch 005/050 | Loss 0.0077 | Win/lose count 4.5/3.0 (1.5)
Epoch 006/050 | Loss 0.0053 | Win/lose count 6

Epoch 032/050 | Loss 0.0235 | Win/lose count 20.0/3.0 (17.0)
Epoch 033/050 | Loss 0.0160 | Win/lose count 8.5/3.0 (5.5)
Epoch 034/050 | Loss 0.0152 | Win/lose count 15.5/3.0 (12.5)
Epoch 035/050 | Loss 0.0333 | Win/lose count 11.0/4.0 (7.0)
Epoch 036/050 | Loss 0.0170 | Win/lose count 14.0/2.0 (12.0)
Epoch 037/050 | Loss 0.0253 | Win/lose count 12.0/4.0 (8.0)
Epoch 038/050 | Loss 0.0183 | Win/lose count 11.0/5.0 (6.0)
Epoch 039/050 | Loss 0.0275 | Win/lose count 9.5/5.0 (4.5)
Epoch 040/050 | Loss 0.0171 | Win/lose count 18.0/5.0 (13.0)
Epoch 041/050 | Loss 0.0137 | Win/lose count 16.5/3.0 (13.5)
Epoch 042/050 | Loss 0.0096 | Win/lose count 13.0/6.0 (7.0)
Epoch 043/050 | Loss 0.0130 | Win/lose count 5.5/2.0 (3.5)
Epoch 044/050 | Loss 0.0202 | Win/lose count 19.0/4.0 (15.0)
Epoch 045/050 | Loss 0.0118 | Win/lose count 17.0/5.0 (12.0)
Epoch 046/050 | Loss 0.0173 | Win/lose count 11.0/4.0 (7.0)
Epoch 047/050 | Loss 0.0109 | Win/lose count 16.5/2.0 (14.5)
Epoch 048/050 | Loss 0.0118 | Win/l

Win/lose count 13.0/1.0. Average score (11.178571428571429)
Win/lose count 11.5/0. Average score (11.189655172413794)
Win/lose count 19.0/0. Average score (11.45)
Win/lose count 17.0/1.0. Average score (11.596774193548388)
Win/lose count 16.5/2.0. Average score (11.6875)
Win/lose count 6.0/1.0. Average score (11.484848484848484)
Win/lose count 21.0/0. Average score (11.764705882352942)
Win/lose count 9.5/1.0. Average score (11.67142857142857)
Win/lose count 3.5/0. Average score (11.444444444444445)
Win/lose count 15.5/1.0. Average score (11.527027027027026)
Win/lose count 25.0/1.0. Average score (11.855263157894736)
Win/lose count 22.0/0. Average score (12.115384615384615)
Win/lose count 15.0/0. Average score (12.1875)
Win/lose count 10.0/2.0. Average score (12.085365853658537)
Win/lose count 16.5/1.0. Average score (12.166666666666666)
Win/lose count 7.0/2.0. Average score (12.0)
Win/lose count 12.5/0. Average score (12.011363636363637)
Win/lose count 10.5/0. Average score (11.9777777

...

Epoch 045/050 | Loss 0.0073 | Win/lose count 6.0/1.0 (5.0)
Epoch 046/050 | Loss 0.0076 | Win/lose count 10.0/2.0 (8.0)
Epoch 047/050 | Loss 0.0026 | Win/lose count 13.5/3.0 (10.5)
Epoch 048/050 | Loss 0.0044 | Win/lose count 9.0/6.0 (3.0)
Epoch 049/050 | Loss 0.0050 | Win/lose count 9.5/4.0 (5.5)
Win/lose count 0/0. Average score (0.0)
Win/lose count 2.5/0. Average score (1.25)
Win/lose count 1.0/0. Average score (1.1666666666666667)
Win/lose count 0/0. Average score (0.875)
Win/lose count 1.0/0. Average score (0.9)
Win/lose count 1.0/0. Average score (0.9166666666666666)
Win/lose count 3.5/0. Average score (1.2857142857142858)
Win/lose count 2.0/1.0. Average score (1.25)
Win/lose count 0.5/0. Average score (1.1666666666666667)
Win/lose count 6.5/0. Average score (1.7)
Win/lose count 1.5/0. Average score (1.6818181818181819)
Win/lose count 1.0/0. Average score (1.625)
Win/lose count 11.0/0. Average score (2.3461538461538463)
Win/lose count 6.0/0. Average score (2.607142857142857)
Win/l

Win/lose count 0.5/0. Average score (3.63953488372093)
Win/lose count 2.5/0. Average score (3.6136363636363638)
Win/lose count 4.5/1.0. Average score (3.611111111111111)
Win/lose count 11.5/2.0. Average score (3.739130434782609)
Win/lose count 4.0/1.0. Average score (3.723404255319149)
Win/lose count 1.0/0. Average score (3.6666666666666665)
Win/lose count 2.0/0. Average score (3.63265306122449)
Win/lose count 0.5/0. Average score (3.57)
Final score: 3.57
saving result {'architecture': 'fc', 'temperature': 0.4, 'score': 3.57}
Epoch 000/050 | Loss 0.0007 | Win/lose count 1.0/2.0 (-1.0)
Epoch 001/050 | Loss 0.0059 | Win/lose count 3.5/4.0 (-0.5)
Epoch 002/050 | Loss 0.0029 | Win/lose count 6.5/2.0 (4.5)
Epoch 003/050 | Loss 0.0173 | Win/lose count 1.5/4.0 (-2.5)
Epoch 004/050 | Loss 0.0143 | Win/lose count 5.5/5.0 (0.5)
Epoch 005/050 | Loss 0.0020 | Win/lose count 3.0/3.0 (0.0)
Epoch 006/050 | Loss 0.0148 | Win/lose count 3.5/2.0 (1.5)
Epoch 007/050 | Loss 0.0062 | Win/lose count 4.5/5.0

Epoch 033/050 | Loss 0.0059 | Win/lose count 18.5/3.0 (15.5)
Epoch 034/050 | Loss 0.0101 | Win/lose count 23.5/3.0 (20.5)
Epoch 035/050 | Loss 0.0048 | Win/lose count 19.0/6.0 (13.0)
Epoch 036/050 | Loss 0.0668 | Win/lose count 16.5/1.0 (15.5)
Epoch 037/050 | Loss 0.0531 | Win/lose count 17.5/4.0 (13.5)
Epoch 038/050 | Loss 0.0584 | Win/lose count 12.0/3.0 (9.0)
Epoch 039/050 | Loss 0.0032 | Win/lose count 14.5/1.0 (13.5)
Epoch 040/050 | Loss 0.0147 | Win/lose count 8.0/3.0 (5.0)
Epoch 041/050 | Loss 0.0060 | Win/lose count 16.5/3.0 (13.5)
Epoch 042/050 | Loss 0.0072 | Win/lose count 37.0/6.0 (31.0)
Epoch 043/050 | Loss 0.0022 | Win/lose count 9.0/5.0 (4.0)
Epoch 044/050 | Loss 0.0037 | Win/lose count 27.0/2.0 (25.0)
Epoch 045/050 | Loss 0.0067 | Win/lose count 22.5/2.0 (20.5)
Epoch 046/050 | Loss 0.0068 | Win/lose count 28.5/8.0 (20.5)
Epoch 047/050 | Loss 0.0050 | Win/lose count 27.0/4.0 (23.0)
Epoch 048/050 | Loss 0.0045 | Win/lose count 10.5/7.0 (3.5)
Epoch 049/050 | Loss 0.0048 | 

Win/lose count 18.0/0. Average score (12.142857142857142)
Win/lose count 2.5/0. Average score (11.810344827586206)
Win/lose count 7.5/1.0. Average score (11.633333333333333)
Win/lose count 4.0/0. Average score (11.387096774193548)
Win/lose count 9.0/0. Average score (11.3125)
Win/lose count 5.5/0. Average score (11.136363636363637)
Win/lose count 14.5/0. Average score (11.235294117647058)
Win/lose count 13.5/0. Average score (11.3)
Win/lose count 13.0/1.0. Average score (11.319444444444445)
Win/lose count 15.0/2.0. Average score (11.364864864864865)
Win/lose count 12.0/0. Average score (11.381578947368421)
Win/lose count 16.0/0. Average score (11.5)
Win/lose count 3.5/0. Average score (11.3)
Win/lose count 2.5/0. Average score (11.085365853658537)
Win/lose count 6.5/0. Average score (10.976190476190476)
Win/lose count 22.0/0. Average score (11.232558139534884)
Win/lose count 7.0/0. Average score (11.136363636363637)
Win/lose count 20.5/2.0. Average score (11.3)
Win/lose count 19.5/0. A

Epoch 017/050 | Loss 0.0061 | Win/lose count 21.0/3.0 (18.0)
Epoch 018/050 | Loss 0.0081 | Win/lose count 12.5/1.0 (11.5)
Epoch 019/050 | Loss 0.0034 | Win/lose count 12.0/0 (12.0)
Epoch 020/050 | Loss 0.0041 | Win/lose count 21.5/1.0 (20.5)
Epoch 021/050 | Loss 0.0060 | Win/lose count 12.5/0 (12.5)
Epoch 022/050 | Loss 0.0032 | Win/lose count 34.0/2.0 (32.0)
Epoch 023/050 | Loss 0.0080 | Win/lose count 21.5/1.0 (20.5)
Epoch 024/050 | Loss 0.0050 | Win/lose count 41.0/4.0 (37.0)
Epoch 025/050 | Loss 0.0030 | Win/lose count 34.0/0 (34.0)
Epoch 026/050 | Loss 0.0026 | Win/lose count 48.5/4.0 (44.5)
Epoch 027/050 | Loss 0.0024 | Win/lose count 40.0/3.0 (37.0)
Epoch 028/050 | Loss 0.0044 | Win/lose count 19.0/1.0 (18.0)
Epoch 029/050 | Loss 0.0046 | Win/lose count 51.0/0 (51.0)
Epoch 030/050 | Loss 0.0044 | Win/lose count 44.5/1.0 (43.5)
Epoch 031/050 | Loss 0.0046 | Win/lose count 54.0/1.0 (53.0)
Epoch 032/050 | Loss 0.0030 | Win/lose count 32.5/2.0 (30.5)
Epoch 033/050 | Loss 0.0040 | Wi

In [41]:
results_df = pd.DataFrame(results)

results_df.head(30)

,architecture,score,temperature
0,cnn,0.42,0.1
1,cnn,1.66,0.2
2,cnn,3.89,0.3
3,cnn,3.67,0.4
4,cnn,5.96,0.5
5,cnn,12.32,0.6
6,cnn,18.16,0.7
7,cnn,37.01,0.8
8,cnn,28.31,0.9
9,fc,0.55,0.1


In [42]:
HTML(display_videos('cnn_test10.mp4'))

In [43]:
HTML(display_videos('fc_test10.mp4'))

**Answer**

> The general observation is that the convolutional models produce better scores than the fully-connected models.

> The main issue that we can observe in the videos is that quite often the rat gets stuck in a small region of the map, precluding it from exploring the entire map. This happens specially when the rat enters a region from which it cannot see any positive reward. This happens sooner on grids with lower temperatures.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [ ]:
def train_explore(agent,env,epoch,prefix=''):
    pass
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        pass
    
## use those samples of code:
#In train explore:
state, reward, game_over = env.act(action, train=True)

## In Environment exploring:
# You will have to change n_state to 3 because you will use one more layer!
reward = 0
if train:
    reward = -self.malus_position[self.x, self.y]
self.malus_position[self.x, self.y] = 0.1

reward = reward + self.board[self.x, self.y]
# 3 "feature" states instead of 2
state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

In [ ]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

In [ ]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***